<a href="https://colab.research.google.com/github/cshmzin/nlp-code/blob/main/Bert_test_MASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install transformers tqdm boto3 requests regex -q
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定为中文
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

In [26]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [27]:
#创建一个需要预测的语句
text = "[CLS] 中国的首都是 [MASK] 京"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(text)
print(tokens)
print(ids)

[CLS] 中国的首都是 [MASK] 京
['[CLS]', '中', '国', '的', '首', '都', '是', '[MASK]', '京']
[101, 704, 1744, 4638, 7674, 6963, 3221, 103, 776]


In [28]:
from transformers import BertForMaskedLM
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

tokens_tensor = torch.tensor([ids])
segments_tensors = torch.zeros_like(tokens_tensor)#构造输出(全为0,格式与输入一样)

maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0] # (1, seq_len, num_hidden_dim)
del maskedLM_model

masked_index = 7
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

for p,t in zip(probs,predicted_tokens):
  print(f"{t}:{int(p*100)}%")


北:90%
南:9%
东:0%


In [29]:
import re
final_text = re.sub('\[MASK\]',predicted_tokens[0],text)
print(final_text)

[CLS] 中国的首都是 北 京
